In [6]:
import ddlpy
import datetime as dt
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.close("all")
import os
import argparse

# parser = argparse.ArgumentParser(description="Download RWS wave data for a given year.")
# parser.add_argument("year", type=int, help="Year to download data for")
# args = parser.parse_args()
# year = args.year
year = 2021
name = str(year)
start_date = dt.datetime(year, 1, 1)  # Year, Month, Day
end_date = dt.datetime(year, 12, 31)  # Year, Month, Day




In [7]:
# Set the path to save the data
main_path = '/gpfs/work3/0/ai4nbs/hurry_wave/north_sea/01_data/Waterinfo_RWS'
output_path = os.path.join(main_path, name)
if not os.path.exists(output_path):
    os.makedirs(output_path)


# Get locations available
locations = ddlpy.locations()




# Get all locations with wave data
bool_grootheid = locations['Grootheid.Omschrijving'].str.contains('golf')



In [11]:
# Find these locations
targets = ['north cormorant', 'Platform F16-A', 'K13a platform', 'Platform Hoorn Q1-A','Ijmuiden munitiestortplaats', 'Euro platform', 'Platform D15-A']

# Apply filters, first on the wave data, then on the names
wave_data_locs = locations[bool_grootheid]
wave_data_locs = wave_data_locs[wave_data_locs['Naam'].str.lower().isin([name.lower() for name in targets])]

print(wave_data_locs['Grootheid.Code'])

Code
NORTHCMRT      Hm0
K13APFM       Tmax
EURPFM        T1/3
D15            Hm0
IJMDMNTSPS    Tmax
F16           Hmax
F16            Hm0
NORTHCMRT     S0BH
K13APFM        Hm0
K13APFM       T1/3
EURPFM        Tmax
EURPFM        Hmax
Q1            Tmax
Q1            T1/3
Q1             Hm0
IJMDMNTSPS    Hmax
K13APFM       S0BH
EURPFM        S0BH
D15           T1/3
IJMDMNTSPS     Hm0
F16           Tmax
NC1            Hm0
NORTHCMRT     Tmax
D15           Hmax
IJMDMNTSPS    T1/3
F16           T1/3
NORTHCMRT      Th0
D15           Tmax
Q1             Th0
Q1            S0BH
NORTHCMRT     Hmax
K13APFM       Hmax
EURPFM         Th0
IJMDMNTSPS     Th0
IJMDMNTSPS    S0BH
K13APFM        Th0
EURPFM         Hm0
Q1            Hmax
NC1           T1/3
Name: Grootheid.Code, dtype: object


In [12]:
print(wave_data_locs['Grootheid.Omschrijving'])

Code
NORTHCMRT       Significante golfhoogte in het spectrale domein
K13APFM                 Maximale golfperiode in het tijdsdomein
EURPFM        Gem. golfperiode langste 1/3 deel v.d. golven ...
D15             Significante golfhoogte in het spectrale domein
IJMDMNTSPS              Maximale golfperiode in het tijdsdomein
F16                      Maximale golfhoogte in het tijdsdomein
F16             Significante golfhoogte in het spectrale domein
NORTHCMRT     Gemiddelde golfrichtingspreiding in het spectr...
K13APFM         Significante golfhoogte in het spectrale domein
K13APFM       Gem. golfperiode langste 1/3 deel v.d. golven ...
EURPFM                  Maximale golfperiode in het tijdsdomein
EURPFM                   Maximale golfhoogte in het tijdsdomein
Q1                      Maximale golfperiode in het tijdsdomein
Q1            Gem. golfperiode langste 1/3 deel v.d. golven ...
Q1              Significante golfhoogte in het spectrale domein
IJMDMNTSPS               Maximale g